<a href="https://colab.research.google.com/github/dltmdgh579/FactCheck_Service/blob/master/SNU_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import boto3
import sys
from gensim.models import FastText

# ==============================================================
# 승호 S3로부터 CSV 파일 불러오기.
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

object_key2 = 'SNU_FastText.csv'
csv_obj2 = client.get_object(Bucket=bucket_name, Key=object_key2)
body = csv_obj2['Body']
csv_string2 = body.read().decode('cp949')

df = pd.read_csv(StringIO(csv_string))
df_train = pd.read_csv(StringIO(csv_string2))

In [38]:
# !pip install boto3
# !pip install konlpy

In [39]:
df_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.735874,0.827440,0.874341,0.783586,0.844526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.769516,0.676043,0.784835,0.848907,0.823045,0.678149,0.793881,0.784963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.686329,0.601476,0.801035,0.799808,0.702206,0.810088,0.768000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.736442,0.725607,0.861988,0.834142,0.736442,0.694960,0.825222,0.829582,0.773539,0.829620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.803000,0.878301,0.724934,0.787956,0.790462,0.736442,0.889945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0.573910,0.774513,0.889347,0.843759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,0.347887,0.842737,0.423220,0.864676,0.809896,0.699228,0.883969,0.781040,0.806430,0.733644,0.85925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,0.381778,0.875480,0.782694,0.703270,0.698105,0.922059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,0.834072,0.845955,0.770608,0.785637,0.803319,0.862845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_train = df_train.fillna(0)
df3 = df2.mul(1000)
df_1000 = df3.astype(int)
df_1000

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,735,827,874,783,844,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,769,676,784,848,823,678,793,784,0,0,0,0,0,0,0,0,0,0,0,0,0
2,686,601,801,799,702,810,768,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,736,725,861,834,736,694,825,829,773,829,0,0,0,0,0,0,0,0,0,0,0
4,802,878,724,787,790,736,889,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,573,774,889,843,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
426,347,842,423,864,809,699,883,781,806,733,859,0,0,0,0,0,0,0,0,0,0
427,381,875,782,703,698,922,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
428,834,845,770,785,803,862,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
df_list = df_train.values.tolist()
# df7 = df_list.remove(0)
df5 = np.array(df_list)

# df2__ = df4.values
# df2__.shape
# df2___ = df2__.reshape(df2__.shape[0], 21, 1)
# df2___.shape

# df8 = []
# for i in range(len(df_list)):
#     df7 = np.delete(df5[i], np.where((df5[i] == 0)))
#     df8.append(df7)
# df8

# df8 = []
for i in range(len(df_list)):
    for a in range(len(df_list[i])):
        try:
            df_list[i].remove(0)
        except:
            pass

df_list
df6 = pad_sequences(df_list, maxlen = 30)
df5

array([[0.73587392, 0.82743951, 0.87434061, ..., 0.        , 0.        ,
        0.        ],
       [0.76951563, 0.6760429 , 0.78483542, ..., 0.        , 0.        ,
        0.        ],
       [0.68632855, 0.60147607, 0.80103497, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.38177839, 0.87548008, 0.78269408, ..., 0.        , 0.        ,
        0.        ],
       [0.83407203, 0.84595503, 0.77060791, ..., 0.        , 0.        ,
        0.        ],
       [0.83727753, 0.73988557, 0.75489187, ..., 0.        , 0.        ,
        0.        ]])

In [43]:
y_train = np.array(df['label'])

In [44]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [48]:
model = Sequential()
model.add(Embedding(1000, 300))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [46]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [47]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(df5, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
6/6 [==============================] - 3s 128ms/step - loss: 0.6927 - acc: 0.5233 - val_loss: 0.7787 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to best_model.h5
Epoch 2/15
6/6 [==============================] - 0s 46ms/step - loss: 0.6911 - acc: 0.5698 - val_loss: 0.7671 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000
Epoch 3/15
6/6 [==============================] - 0s 45ms/step - loss: 0.6863 - acc: 0.5698 - val_loss: 0.7822 - val_acc: 0.0000e+00

Epoch 00003: val_acc did not improve from 0.00000
Epoch 4/15
6/6 [==============================] - 0s 44ms/step - loss: 0.6860 - acc: 0.5698 - val_loss: 0.7848 - val_acc: 0.0000e+00

Epoch 00004: val_acc did not improve from 0.00000
Epoch 5/15
6/6 [==============================] - 0s 44ms/step - loss: 0.6849 - acc: 0.5698 - val_loss: 0.7903 - val_acc: 0.0000e+00

Epoch 00005: val_acc did not improve from 0.00000
Epoch 6/15
6/6 [==============================]